# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096878


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:03,  2.35s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.72s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.26s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:20,  1.11it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.85it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.83it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  3.99it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  3.50it/s]

pinknebula1543                        0.003625
not-logged-in-8cf2e6929490167ca6ab    0.018966
jnarayanbvg                           0.000405
Trollio12345                          0.048933
Jonas_Cross                           0.000500
not-logged-in-a0d39065895c84f0fd2d    0.001104
Nauter                                0.001383
AaronWallage117                       0.075055
CScampstudent                         0.003020
Enfysh                                0.022642
ElisabethB                            0.134218
sn346808                              0.033717
isacami2020                           0.003806
Rmarks1701                            0.003450
not-logged-in-14d09f70263c1fe15cc3    0.000392
LeeJoseph                             0.013081
thisguyiknow                          0.000432
Sylverone                             0.011557
djkv                                  0.001344
ElieB                                 0.004708
ccarles.100                           0.004055
KevinKosa    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())